# Tune Tutorial

<img src="tune.png" alt="Tune Logo" width="400"/>


Tune is a scalable framework for model training and hyperparameter search with a focus on deep learning and deep reinforcement learning.

**Code**: https://github.com/ray-project/ray/tree/master/python/ray/tune

**Examples**: https://github.com/ray-project/ray/tree/master/python/ray/tune/examples

**Documentation**: http://ray.readthedocs.io/en/latest/tune.html

**Mailing List** https://groups.google.com/forum/#!forum/ray-dev

# Overview

Tuning hyperparameters is often the most expensive part of the machine learning workflow. Tune is built to address this, demonstrating an efficient and scalable solution for this pain point.


## Outline
This tutorial will walk you through the following process:

1. Creating and training a model on a toy dataset (MNIST)
2. Integrating Tune into your workflow
3. Trying out advanced features - plugging in an efficient scheduler
4. Validating your trained model
5. (Optional) Try out a search algorithm


In [1]:
from helper import *
import numpy as np
from IPython.display import HTML
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

limit_threads(4)
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


## PART 1: Creating a model to be trained.

Let's create a Convolutional Neural Network model. Convolutional Neural Networks (ConvNets or CNNs) are a category of Neural Networks that have proven very effective in areas such as image recognition and classification. The details of how a Convolutional Neural Network works are unimportant here, but you're welcome to read more about them here: http://cs231n.github.io/convolutional-networks/

<img src="cnn.png" alt="MNIST Visualization" width="800"/>


This convolutional neural network model will be used classify digits (from the MNIST dataset).

<img src="mnist.png" alt="MNIST Visualization" width="400"/>

This is a fairly simple dataset, but it enables us to explore Tune's functionality in depth.
We will use 60,000 images to train the network. The images are 28x28 NumPy arrays, with pixel values ranging between 0 and 255. The labels are an array of integers, ranging from 0 to 9. These correspond to the digit the image represents.

Here, we'll specify some arguments and some reasonable defaults for this model. These are the hyperparameters settings that we will later use to further optimize this model.

In [2]:
import argparse
parser = argparse.ArgumentParser(description='Keras MNIST Example')
parser.add_argument('--lr', type=float, default=0.1, help='learning rate')
parser.add_argument('--momentum', type=float, default=0.0, help='SGD momentum')
parser.add_argument('--kernel1', type=int, default=3, help='Size of first kernel')
parser.add_argument('--kernel2', type=int, default=3, help='Size of second kernel')
parser.add_argument('--poolsize', type=int, default=2, help='Size of Poolin')
parser.add_argument('--dropout1', type=float, default=0.25, help='Size of first kernel')
parser.add_argument('--hidden', type=int, default=4, help='Size of Hidden Layer')
parser.add_argument('--dropout2', type=float, default=0.5, help='Size of first kernel')

DEFAULT_ARGS = vars(parser.parse_known_args()[0])

This below function will create and return a Convolutional Neural Network. You don't need to modify this function

In [3]:
def make_model(parameters):
    config = DEFAULT_ARGS.copy()  # This is obtained via the global scope
    config.update(parameters)
    num_classes = 10
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(config["kernel1"], config["kernel1"]),
                     activation='relu', input_shape=(28, 28, 1)))
    model.add(Conv2D(64, (config["kernel2"], config["kernel2"]), activation='relu'))
    model.add(MaxPooling2D(pool_size=(config["poolsize"], config["poolsize"])))
    model.add(Dropout(config["dropout1"]))
    model.add(Flatten())
    model.add(Dense(config["hidden"], activation='relu'))
    model.add(Dropout(config["dropout2"]))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.SGD(
                      lr=config["lr"], momentum=config["momentum"]),
                  metrics=['accuracy'])
    return model

### Exercise: Setup a basic model training script.

The process of training the neural network model occurs as follows:

1. Feed the training data to the model—in this example, the `batch_of_data` and `batch_of_labels` arrays.
2. The model learns to associate images and labels.

**Exercise**: Finish the TODOs below. Here are a few hints:

1) `data_generator` is an iterator that returns (`batch_of_data`, `batch_of_labels`), like follows:

```python
for batch_of_data, batch_of_labels in data_generator:
    do_something_interesting()
```
2) You can use `model.fit(batch_of_data, batch_of_labels)` to repeatedly improve the model.

In [4]:
def train_mnist(args):
    """Loads data, does one pass over the data, and saves the weights."""
    data_generator = load_data()
    model = make_model(args)
    for batch_of_data, batch_of_labels in data_generator:
        model.fit(batch_of_data, batch_of_labels)
    ## TODO: use the `data_generator` to iterate over the data and improve the model
    model.save_weights("./weights.h5")
    return model

Let's run this above training script to make sure things work.

In [5]:
first_model = train_mnist(DEFAULT_ARGS)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
32/32 [==============================] - 0s 7ms/step - loss: 2.3033 - acc: 0.0625
Epoch 1/1
32/32 [==============================] - 0s 938us/step - loss: 2.3477 - acc: 0.1250
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.3003 - acc: 0.0938
Epoch 1/1
32/32 [==============================] - 0s 990us/step - loss: 2.2996 - acc: 0.0625
Epoch 1/1
32/32 [==============================] - 0s 989us/step - loss: 2.3050 - acc: 0.0625
Epoch 1/1
32/32 [==============================] - 0s 950us/step - loss: 2.2844 - acc: 0.1250
Epoch 1/1
32/32 [==============================] - 0s 943us/step - loss: 2.2777 - acc: 0.0938
Epoch 1/1
32/32 [=====================

32/32 [==============================] - 0s 991us/step - loss: 2.1615 - acc: 0.1562
Epoch 1/1
32/32 [==============================] - 0s 984us/step - loss: 2.2282 - acc: 0.1250
Epoch 1/1
32/32 [==============================] - 0s 957us/step - loss: 2.1241 - acc: 0.1875
Epoch 1/1
32/32 [==============================] - 0s 971us/step - loss: 2.1986 - acc: 0.2188
Epoch 1/1
32/32 [==============================] - 0s 989us/step - loss: 2.0869 - acc: 0.1250
Epoch 1/1
32/32 [==============================] - 0s 956us/step - loss: 2.0982 - acc: 0.3125
Epoch 1/1
32/32 [==============================] - 0s 967us/step - loss: 2.2887 - acc: 0.1562
Epoch 1/1
32/32 [==============================] - 0s 992us/step - loss: 2.3429 - acc: 0.0938
Epoch 1/1
32/32 [==============================] - 0s 963us/step - loss: 2.2498 - acc: 0.0625
Epoch 1/1
32/32 [==============================] - 0s 973us/step - loss: 2.1923 - acc: 0.1250
Epoch 1/1
32/32 [==============================] - 0s 998us/step - los

Epoch 1/1
32/32 [==============================] - 0s 990us/step - loss: 2.2196 - acc: 0.2188
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.1652 - acc: 0.1875
Epoch 1/1
32/32 [==============================] - 0s 971us/step - loss: 2.2082 - acc: 0.0938
Epoch 1/1
32/32 [==============================] - 0s 981us/step - loss: 2.2849 - acc: 0.0000e+00
Epoch 1/1
32/32 [==============================] - 0s 1000us/step - loss: 2.3198 - acc: 0.0938
Epoch 1/1
32/32 [==============================] - 0s 946us/step - loss: 2.3277 - acc: 0.0938
Epoch 1/1
32/32 [==============================] - 0s 960us/step - loss: 2.1764 - acc: 0.1250
Epoch 1/1
32/32 [==============================] - 0s 971us/step - loss: 2.2647 - acc: 0.0625
Epoch 1/1
32/32 [==============================] - 0s 965us/step - loss: 2.2366 - acc: 0.0625
Epoch 1/1
32/32 [==============================] - 0s 976us/step - loss: 2.0264 - acc: 0.1562
Epoch 1/1
32/32 [==============================] - 0s 964

Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.1840 - acc: 0.1250
Epoch 1/1
32/32 [==============================] - 0s 996us/step - loss: 2.3119 - acc: 0.0625
Epoch 1/1
32/32 [==============================] - 0s 998us/step - loss: 2.1286 - acc: 0.1562
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.2933 - acc: 0.1250
Epoch 1/1
32/32 [==============================] - 0s 973us/step - loss: 2.2528 - acc: 0.1562
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.3001 - acc: 0.1562
Epoch 1/1
32/32 [==============================] - 0s 960us/step - loss: 2.2804 - acc: 0.1250
Epoch 1/1
32/32 [==============================] - 0s 941us/step - loss: 2.1362 - acc: 0.1250
Epoch 1/1
32/32 [==============================] - 0s 968us/step - loss: 2.0813 - acc: 0.1250
Epoch 1/1
32/32 [==============================] - 0s 979us/step - loss: 2.3184 - acc: 0.1250
Epoch 1/1
32/32 [==============================] - 0s 962us/step -

Epoch 1/1
32/32 [==============================] - 0s 966us/step - loss: 2.3158 - acc: 0.0312
Epoch 1/1
32/32 [==============================] - 0s 955us/step - loss: 2.3002 - acc: 0.0938
Epoch 1/1
32/32 [==============================] - 0s 972us/step - loss: 2.1937 - acc: 0.2188
Epoch 1/1
32/32 [==============================] - 0s 952us/step - loss: 2.2721 - acc: 0.0312
Epoch 1/1
32/32 [==============================] - 0s 970us/step - loss: 2.2953 - acc: 0.0938
Epoch 1/1
32/32 [==============================] - 0s 942us/step - loss: 2.2064 - acc: 0.1562
Epoch 1/1
32/32 [==============================] - 0s 943us/step - loss: 2.0860 - acc: 0.1562
Epoch 1/1
32/32 [==============================] - 0s 925us/step - loss: 2.2226 - acc: 0.0938
Epoch 1/1
32/32 [==============================] - 0s 946us/step - loss: 2.2181 - acc: 0.1250
Epoch 1/1
32/32 [==============================] - 0s 961us/step - loss: 2.2476 - acc: 0.0938
Epoch 1/1
32/32 [==============================] - 0s 966us/

32/32 [==============================] - 0s 961us/step - loss: 2.1348 - acc: 0.1875
Epoch 1/1
32/32 [==============================] - 0s 975us/step - loss: 2.2888 - acc: 0.0938
Epoch 1/1
32/32 [==============================] - 0s 1000us/step - loss: 2.2071 - acc: 0.1562
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.3678 - acc: 0.1562
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.2999 - acc: 0.0625
Epoch 1/1
32/32 [==============================] - 0s 996us/step - loss: 2.2465 - acc: 0.0625
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.2274 - acc: 0.2188
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.2122 - acc: 0.2188
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.2338 - acc: 0.1250
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.1102 - acc: 0.2188
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.2167 - a

Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.2422 - acc: 0.1250
Epoch 1/1
32/32 [==============================] - 0s 992us/step - loss: 2.2333 - acc: 0.2500
Epoch 1/1
32/32 [==============================] - 0s 985us/step - loss: 2.2066 - acc: 0.1562
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.2931 - acc: 0.0312
Epoch 1/1
32/32 [==============================] - 0s 955us/step - loss: 1.9793 - acc: 0.2188
Epoch 1/1
32/32 [==============================] - 0s 938us/step - loss: 2.1919 - acc: 0.1250
Epoch 1/1
32/32 [==============================] - 0s 960us/step - loss: 2.3005 - acc: 0.2188
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.1759 - acc: 0.2812
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.1645 - acc: 0.1250
Epoch 1/1
32/32 [==============================] - 0s 983us/step - loss: 2.2891 - acc: 0.0938
Epoch 1/1
32/32 [==============================] - 0s 989us/step - l

In [ ]:
evaluate(first_model)

Let's now quickly try out this model to see if it works as expected. We'll load the model with our trained weights.

**Exercise**: Try to write a digit into the box below. This will automatically save your input in a variable `data` behind the scenes.

In [ ]:
data = None
HTML(open("input.html").read())

(tip: don't expect it to work)

In [ ]:
prepared_data = prepare_data(data)
print("This model predicted your input as", first_model.predict(prepared_data).argmax())

**You've now set up a model that we can use Tune to optimize!**

## Part 2: Setting up Tune

One thing we might want to do now is find better hyperparameters so that our model trains more quickly and possibly to a higher accuracy. Let's make some minor modifications to utilize Tune. 

Tune uses Ray as a backend, so we will first import and initialize Ray.

In [9]:
import ray
from ray import tune

ray.init(ignore_reinit_error=True)

2019-04-04 13:39:00,913	WARNING worker.py:1406 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-04-04 13:39:00,914	INFO node.py:423 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-04_13-39-00_51359/logs.
2019-04-04 13:39:01,029	INFO services.py:363 -- Waiting for redis server at 127.0.0.1:34822 to respond...
2019-04-04 13:39:01,144	INFO services.py:363 -- Waiting for redis server at 127.0.0.1:60077 to respond...
2019-04-04 13:39:01,149	INFO services.py:760 -- Starting Redis shard with 6.87 GB max memory.
2019-04-04 13:39:01,163	INFO services.py:1384 -- Starting the Plasma object store with 10.31 GB memory using /tmp.


{'node_ip_address': None,
 'object_store_address': '/tmp/ray/session_2019-04-04_13-39-00_51359/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-04-04_13-39-00_51359/sockets/raylet',
 'redis_address': '10.0.0.185:34822',
 'webui_url': None}

Tune will automate and distribute your hyperparameter search by scheduling a number of **trials** on a machine. Each trial runs a user-defined Python function with a sampled set of hyperparameters. 

### Exercise: Two steps to use Tune:

Step 1) For the function you wish to tune, we need to change the signature to a specific format as shown below. Specifically: **pass in a ``reporter`` object to the below `train_mnist_tune` class**.

```python
def trainable(config, reporter):
    """
    Args:
        config (dict): Parameters provided from the search algorithm
            or variant generation.
        reporter (Reporter): Handle to report intermediate metrics to Tune.
    """
```

Step 2) We want to keep track of performance as the model is training. Specifically: **get the `mean_accuracy` from Keras, and call the ``reporter`` to report the `mean_accuracy` for every batch**. 

You can get model accuracy from Keras with the following code:

```python
mean_accuracy = model.evaluate(x_batch, y_batch)[1]
```


Example of using the reporter:

```python
def train_func(config, reporter):  # add a reporter arg
    # ...
    for data, target in dataset:
        model.fit(data, target)
        # save model here
        accuracy = model.evaluate(x_batch, y_batch)[1]
        reporter(mean_accuracy=accuracy) # report metrics
```


In [6]:
def train_mnist_tune(config, reporter): ### TODO: Change this function signature following step 1 #####
    data_generator = load_data()
    model = make_model(config)
    for i, (x_batch, y_batch) in enumerate(data_generator):
        model.fit(x_batch, y_batch, verbose=0)
        if i % 3 == 0:
            last_checkpoint = "weights_tune_{}.h5".format(i)
            model.save_weights(last_checkpoint)
        ### Don't change above ############### 
        mean_accuracy = model.evaluate(x_batch, y_batch)[1]
        reporter(mean_accuracy=mean_accuracy, timesteps_total=i, checkpoint=last_checkpoint)
        ### TODO: Use the reporter here to fill out intermediate metrics following step 2###

In [7]:
# This may take 30 seconds or so to run if incorrectly written
assert test_reporter(train_mnist_tune)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
32/32 [==============================] - 0s 2ms/step
Works!


**If you've done this correctly, you now have properly converted your function to be Tune-compatible!**

### Exercise: Let's now try to search over some parameters. 

*NOTE: You can find the documentation for this section here: https://ray.readthedocs.io/en/latest/tune-usage.html#specifying-experiments*

In this section, we'll use some basic Tune features for training - namely specifying a stopping criteria and a search space. 

Let's first create a Tune Experiment specification. The relevant documentation for the Experiment class is here:

```python
class ray.tune.Experiment(name, run, stop=None, config=None, ... ):
    """Tracks experiment specifications.

    Parameters:
        name (str): Name of experiment.
        run (function|class|str): The algorithm or model to train.
            This may refer to the name of a built-on algorithm
            (e.g. RLLib's DQN or PPO), a user-defined trainable
            function or class, or the string identifier of a
            trainable function or class registered in the tune registry.
        stop (dict): The stopping criteria. The keys may be any field in
            the return result of 'train()', whichever is reached first.
            Defaults to empty dict.
        config (dict): Algorithm-specific configuration for Tune variant
            generation (e.g. env, hyperparams). Defaults to empty dict.
            Custom search algorithms may ignore this.
        trial_resources (dict): Machine resources to allocate per trial,
            e.g. ``{"cpu": 64, "gpu": 8}``. Note that GPUs will not be
            assigned unless you specify them here. Defaults to 1 CPU and 0
            GPUs in ``Trainable.default_resource_request()``.
        ...
```

**Part 1**: First, **set the stopping criteria to when `mean_accuracy` passes `0.95`**. For example, to specify that trials will be stopped whenever they report `arbitrary_metric` that is `>= 500`, do:

```python
stop={"arbitrary_metric": 500}
```


**Part 2**: We also want to designate a search space. We'll search over *learning rate*, which sets the step size of our model update, and *momentum*, which helps accelerate gradients vectors in the right directions, thus leading to faster converging.

You can use `tune.grid_search` to specify an axis of a grid search. By default, Tune also supports sampling parameters from user-specified lambda functions, which can be used independently or in combination with grid search.  The following example shows grid search over a set of values combined with random sampling from a lambda functions, generating 3 different trials. 

```python
configuration = tune.Experiment(
    # ...
    config={
        "arbitrary_parameter1": tune.sample_from(lambda spec: np.random.uniform(0.1, 100)),
        "arbitrary_parameter2": tune.grid_search([16, 64, 256]),
        # ...
    }
)
```

Specifically, 
1. randomly search for learning rate `"lr"` between 0.001 to 0.1,
2. do a grid search over `"momentum"` for `[0.2, 0.4, 0.6]` 

In [12]:
configuration = tune.Experiment(
    "experiment_name",
    run=train_mnist_tune,
    resources_per_trial={"cpu": 4},
    stop={"mean_accuracy": 0.95},  # TODO: Part 1
    config={"lr": tune.sample_from(lambda spec: np.random.uniform(0.001, 0.1)),
            "momentum": tune.grid_search([0.2, 0.4, 0.6])}  # TODO: Part 2
)

assert configuration.spec.get("stop", {}).get("mean_accuracy") == 0.95
assert "grid_search" in configuration.spec.get("config", {}).get("momentum", {})
assert "lr" in configuration.spec.get("config", {})
print("Success!")

Success!


Now, we can run our experiment with a single line of code. 

*Note*: Be sure pay attention to the `acc` metric next to each running trial. That indicates the most recently reported mean accuracy for that trial. This should evaluate in less than a minute. The output will look something similar to:

```
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 8/8 CPUs, 0/1 GPUs
Result logdir: .../ray_results/experiment_name
RUNNING trials:
 - train_mnist_tune_0_lr=0.085836,momentum=0.2:	RUNNING [pid=44320], 4 s, 3 iter, 0.406 acc
 - train_mnist_tune_1_lr=0.062562,momentum=0.4:	RUNNING [pid=44321], 3 s, 2 iter, 0.219 acc
 - train_mnist_tune_2_lr=0.099461,momentum=0.6:	RUNNING [pid=44317], 3 s, 2 iter, 0.281 acc
 ```

In [13]:
import sys
# sys.stdout
trials = tune.run_experiments(configuration, verbose=False)

2019-04-04 13:43:05,121	INFO tune.py:60 -- Tip: to resume incomplete experiments, pass resume='prompt' or resume=True to run()
2019-04-04 13:43:05,122	INFO tune.py:211 -- Starting a new experiment.
2019-04-04 13:43:05,391	WARNING util.py:62 -- The `start_trial` operation took 0.25650501251220703 seconds to complete, which may be a performance bottleneck.


(pid=51640) Using TensorFlow backend.
(pid=51634) Using TensorFlow backend.
(pid=51638) Using TensorFlow backend.
(pid=51638) x_train shape: (60000, 28, 28, 1)
(pid=51638) 60000 train samples
(pid=51638) 10000 test samples
(pid=51634) x_train shape: (60000, 28, 28, 1)
(pid=51634) 60000 train samples
(pid=51634) 10000 test samples
(pid=51640) x_train shape: (60000, 28, 28, 1)
(pid=51640) 60000 train samples
(pid=51640) 10000 test samples
(pid=51634) WARNING:tensorflow:From /anaconda3/lib/python3.5/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
(pid=51634) Instructions for updating:
(pid=51634) Colocations handled automatically by placer.
(pid=51638) WARNING:tensorflow:From /anaconda3/lib/python3.5/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future vers

(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 544us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 1ms/step
(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 505us/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 261us/step
(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 710us/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 406us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 527us/step
(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 354us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 360us/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 364us/step
(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 360us/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 413us/step
(pid=51640) 
(pid=51640) 32/32 [==========

(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 608us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 644us/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 506us/step
(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 373us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 632us/step
(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 685us/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 656us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 549us/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 607us/step
(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 406us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 484us/step
(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 445us/step
(pid=51638) 
(pid=51638) 32/32 [========

(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 555us/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 621us/step
(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 617us/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 352us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 913us/step
(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 234us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 603us/step
(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 686us/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 400us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 522us/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 406us/step
(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 362us/step
(pid=51640) 
(pid=51640) 32/32 [========

...

(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 568us/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 727us/step
(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 954us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 382us/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 843us/step
(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 559us/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 402us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 394us/step
(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 885us/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 387us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 560us/step
(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 452us/step
(pid=51640) 
(pid=51640) 32/32 [========

2019-04-04 13:44:03,437	INFO ray_trial_executor.py:178 -- Destroying actor for trial train_mnist_tune_1_lr=0.01906,momentum=0.4. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 353us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 503us/step
(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 887us/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 663us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 492us/step
(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 864us/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 345us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 527us/step
(pid=51634) 
(pid=51634) 32/32 [==============================] - 0s 1ms/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 429us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 484us/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 380us/step
(pid=51640) 
(pid=51640) 32/32 [==========

2019-04-04 13:44:03,781	INFO ray_trial_executor.py:178 -- Destroying actor for trial train_mnist_tune_0_lr=0.0065367,momentum=0.2. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2019-04-04 13:44:03,861	INFO ray_trial_executor.py:178 -- Destroying actor for trial train_mnist_tune_2_lr=0.0024039,momentum=0.6. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 395us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 749us/step
(pid=51638) 
(pid=51638) 32/32 [==============================] - 0s 942us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 363us/step
(pid=51640) 
(pid=51640) 32/32 [==============================] - 0s 354us/step


You can expect the result below to be about `0.6`, although your mileage may vary (and it's OK).

In [14]:
print("The best result is", get_best_result(trials, metric="mean_accuracy"))

The best result is {'mean_accuracy': 0.625}


**You've run your first Tune experiment!**

## Exercise: Try using a scheduler



By default, Tune schedules trials in serial order with the `FIFOScheduler` class. However, you can also specify a custom scheduling algorithm that can early stop trials or perturb parameters. 

Let's use a state of the art algorithm, `HyperBand`, to scale up and accelerate our training. Hyperband is an algorithm that focuses on speeding up random search through adaptive resource allocation and early-stopping.

**Part 1**: Let's scale up our search. 

1) Sample the search space 5 times. (https://ray.readthedocs.io/en/latest/tune-usage.html#sampling-multiple-times).

2) Search over another hyperparameter: `"hidden"` from 16 to 512 which specifies the size of the last neural network layer.

Here, use `np.random.randint`. 
```python
numpy.random.randint(low, high=None, size=None, dtype='l')
    """Return random integers from low (inclusive) to high (exclusive)."""
```

An extended version of the `Experiment` documentation is shown below. Note that you should expect a total of 15 trials, due to the usage of `grid_search`.

```python
class ray.tune.Experiment(name, run, stop=None, config=None, ... ):
    """Tracks experiment specifications.

    Parameters:
        name (str): Name of experiment.
        run (function|class|str): The algorithm or model to train.
            This may refer to the name of a built-on algorithm
            (e.g. RLLib's DQN or PPO), a user-defined trainable
            function or class, or the string identifier of a
            trainable function or class registered in the tune registry.
        stop (dict): The stopping criteria. The keys may be any field in
            the return result of 'train()', whichever is reached first.
            Defaults to empty dict.
        config (dict): Algorithm-specific configuration for Tune variant
            generation (e.g. env, hyperparams). Defaults to empty dict.
            Custom search algorithms may ignore this.
        trial_resources (dict): Machine resources to allocate per trial,
            e.g. ``{"cpu": 64, "gpu": 8}``. Note that GPUs will not be
            assigned unless you specify them here. Defaults to 1 CPU and 0
            GPUs in ``Trainable.default_resource_request()``.
        num_samples (int): Number of times to sample from the
            hyperparameter space. Defaults to 1. If `grid_search` is
            provided as an argument, the grid will be repeated
            `num_samples` of times.
        ...
```

In [19]:
configuration2 = tune.Experiment(
    "experiment2",
    run=train_mnist_tune,
    num_samples=5, ## TODO: Change this to 5
    resources_per_trial={"cpu": 4},
    stop={"mean_accuracy": 0.95},
    config={
        "lr": tune.sample_from(lambda spec: np.random.uniform(0.001, 0.1)),
        "momentum": tune.grid_search([0.2, 0.4, 0.6]),
        "hidden": tune.sample_from(lambda spec: np.random.randint(16, high=513)), ## TODO: Sample uniformly from 16 to 512
    }
)


**Part 2**: Create an Asynchronous HyperBand Scheduler (https://ray.readthedocs.io/en/latest/tune-schedulers.html#asynchronous-hyperband). The documentation is shown below. 

Be sure to set the `time_attr` to `timesteps_total` and `reward_attr` to `mean_accuracy`.

```python
class AsyncHyperBandScheduler(FIFOScheduler):
    """This provides HyperBand functionality to your search.
    
    Hyperband is an algorithm that focuses on speeding up 
    random search through adaptive resource allocation and early-stopping.

    See https://openreview.net/forum?id=S1Y7OOlRZ

    Args:
        time_attr (str): A training result attr to use for comparing time.
            Note that you can pass in something non-temporal such as
            `training_iteration` as a measure of progress, the only requirement
            is that the attribute should increase monotonically.
        reward_attr (str): The training result objective value attribute. As
            with `time_attr`, this may refer to any objective value. Stopping
            procedures will use this attribute.
        ...
        
    Examples:
        >>> hyperband = AsyncHyperBandScheduler(
        >>>     time_attr='timesteps_total',
        >>>     reward_attr='mean_accuracy')
    """

```

In [20]:
from ray.tune.schedulers import AsyncHyperBandScheduler

## TODO: Create an Asynchronous HyperBand Scheduler
hyperband = AsyncHyperBandScheduler(
    time_attr='timesteps_total',
    reward_attr='mean_accuracy')

Given the previous configuration, pass in the HyperBand scheduler to `run_experiments`.

Recall that the `run_experiments` API is:
```python
def run_experiments(experiments=None,
                    search_alg=None,
                    scheduler=None,
                    ...):
    """Runs and blocks until all trials finish.

    Args:
        experiments (Experiment | list | dict): Experiments to run. Will be
            passed to `search_alg` via `add_configurations`.
        search_alg (SearchAlgorithm): Search Algorithm. Defaults to
            BasicVariantGenerator.
        scheduler (TrialScheduler): Scheduler for executing
            the experiment. Choose among FIFO (default), MedianStopping,
            AsyncHyperBand, and HyperBand.
        ...
    
    Returns:
        List of Trial objects, holding data for each executed trial.
```


**Exercise**: Call `run_experiments` with the correct arguments. This may take multiple minutes for the experiment to complete.

In [18]:
# Note that you should be using `configuration2` instead of `configuration`.
trials = tune.run_experiments(configuration2, scheduler=hyperband, verbose=False)

2019-04-04 15:18:00,299	INFO tune.py:60 -- Tip: to resume incomplete experiments, pass resume='prompt' or resume=True to run()
2019-04-04 15:18:00,300	INFO tune.py:211 -- Starting a new experiment.


(pid=51641) Using TensorFlow backend.
(pid=51632) Using TensorFlow backend.
(pid=51635) Using TensorFlow backend.
(pid=51641) x_train shape: (60000, 28, 28, 1)
(pid=51641) 60000 train samples
(pid=51641) 10000 test samples
(pid=51632) x_train shape: (60000, 28, 28, 1)
(pid=51632) 60000 train samples
(pid=51632) 10000 test samples
(pid=51635) x_train shape: (60000, 28, 28, 1)
(pid=51635) 60000 train samples
(pid=51635) 10000 test samples
(pid=51641) WARNING:tensorflow:From /anaconda3/lib/python3.5/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
(pid=51641) Instructions for updating:
(pid=51641) Colocations handled automatically by placer.
(pid=51641) WARNING:tensorflow:From /anaconda3/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future

(pid=51632) 
(pid=51632) 32/32 [==============================] - 0s 698us/step
(pid=51641) 
(pid=51641) 32/32 [==============================] - 0s 1ms/step
(pid=51632) 
(pid=51632) 32/32 [==============================] - 0s 798us/step
(pid=51641) 
(pid=51641) 32/32 [==============================] - 0s 417us/step
(pid=51635) 
(pid=51635) 32/32 [==============================] - 0s 695us/step
(pid=51635) 
(pid=51635) 32/32 [==============================] - 0s 358us/step
(pid=51632) 
(pid=51632) 32/32 [==============================] - 0s 730us/step
(pid=51641) 
(pid=51641) 32/32 [==============================] - 0s 455us/step
(pid=51635) 
(pid=51635) 32/32 [==============================] - 0s 908us/step
(pid=51632) 
(pid=51632) 32/32 [==============================] - 0s 421us/step
(pid=51641) 
(pid=51641) 32/32 [==============================] - 0s 484us/step
(pid=51635) 
(pid=51635) 32/32 [==============================] - 0s 581us/step
(pid=51635) 
(pid=51635) 32/32 [==========

(pid=51632) 
(pid=51632) 32/32 [==============================] - 0s 515us/step
(pid=51641) 
(pid=51641) 32/32 [==============================] - 0s 555us/step
(pid=51635) 
(pid=51635) 32/32 [==============================] - 0s 810us/step
(pid=51632) 
(pid=51632) 32/32 [==============================] - 0s 566us/step
(pid=51641) 
(pid=51641) 32/32 [==============================] - 0s 600us/step
(pid=51635) 
(pid=51635) 32/32 [==============================] - 0s 574us/step
(pid=51641) 
(pid=51641) 32/32 [==============================] - 0s 409us/step
(pid=51632) 
(pid=51632) 32/32 [==============================] - 0s 759us/step
(pid=51635) 
(pid=51635) 32/32 [==============================] - 0s 644us/step
(pid=51641) 
(pid=51641) 32/32 [==============================] - 0s 366us/step
(pid=51632) 
(pid=51632) 32/32 [==============================] - 0s 861us/step
(pid=51635) 
(pid=51635) 32/32 [==============================] - 0s 540us/step
(pid=51641) 
(pid=51641) 32/32 [========

2019-04-04 15:18:08,801	INFO ray_trial_executor.py:178 -- Destroying actor for trial train_mnist_tune_1_hidden=416,lr=0.051892,momentum=0.4. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


...

(pid=52810) 
(pid=52810) 32/32 [==============================] - 0s 392us/step
(pid=52810) 
(pid=52810) 32/32 [==============================] - 0s 449us/step


2019-04-04 15:18:52,794	ERROR trial_runner.py:264 -- Trial Runner checkpointing failed.
Traceback (most recent call last):
  File "/anaconda3/lib/python3.5/site-packages/ray/tune/trial_runner.py", line 262, in step
    self.checkpoint()
  File "/anaconda3/lib/python3.5/site-packages/ray/tune/trial_runner.py", line 158, in checkpoint
    TrialRunner.CKPT_FILE_TMPL.format(self._session_str)))
FileNotFoundError: [Errno 2] No such file or directory: '/Users/georgestoica/ray_results/experiment2/.tmp_checkpoint' -> '/Users/georgestoica/ray_results/experiment2/experiment_state-2019-04-04_15-18-00.json'
2019-04-04 15:18:52,898	ERROR trial_runner.py:264 -- Trial Runner checkpointing failed.
Traceback (most recent call last):
  File "/anaconda3/lib/python3.5/site-packages/ray/tune/trial_runner.py", line 262, in step
    self.checkpoint()
  File "/anaconda3/lib/python3.5/site-packages/ray/tune/trial_runner.py", line 158, in checkpoint
    TrialRunner.CKPT_FILE_TMPL.format(self._session_str)))
Fil

(pid=52810) 
(pid=52810) 32/32 [==============================] - 0s 378us/step
(pid=52810) 
(pid=52810) 32/32 [==============================] - 0s 610us/step
(pid=52810) 
(pid=52810) 32/32 [==============================] - 0s 515us/step
(pid=52810) 
(pid=52810) 32/32 [==============================] - 0s 562us/step
(pid=52810) 
(pid=52810) 32/32 [==============================] - 0s 465us/step
(pid=52810) 
(pid=52810) 32/32 [==============================] - 0s 449us/step
(pid=52810) 
(pid=52810) 32/32 [==============================] - 0s 525us/step
(pid=52810) 
(pid=52810) 32/32 [==============================] - 0s 447us/step


2019-04-04 15:18:53,857	INFO ray_trial_executor.py:178 -- Destroying actor for trial train_mnist_tune_14_hidden=175,lr=0.091013,momentum=0.6. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2019-04-04 15:18:53,868	ERROR trial_runner.py:264 -- Trial Runner checkpointing failed.
Traceback (most recent call last):
  File "/anaconda3/lib/python3.5/site-packages/ray/tune/trial_runner.py", line 262, in step
    self.checkpoint()
  File "/anaconda3/lib/python3.5/site-packages/ray/tune/trial_runner.py", line 158, in checkpoint
    TrialRunner.CKPT_FILE_TMPL.format(self._session_str)))
FileNotFoundError: [Errno 2] No such file or directory: '/Users/georgestoica/ray_results/experiment2/.tmp_checkpoint' -> '/Users/georgestoica/ray_results/experiment2/experiment_state-2019-04-04_15-18-00.json'


(pid=52810) 
(pid=52810) 32/32 [==============================] - 0s 385us/step
(pid=52810) 
(pid=52810) 32/32 [==============================] - 0s 898us/step


**Exercise**: Now, let's get the best model from your search process, and check its validation accuracy compared to the first model we created above.

In [21]:
best_model = get_best_model(make_model, trials, metric="mean_accuracy")

Creating model...
Loading from /Users/georgestoica/ray_results/experiment2/train_mnist_tune_0_hidden=93,lr=0.030402,momentum=0.2_2019-04-04_15-18-00frtol7k2/weights_tune_87.h5


In [22]:
print("Best model...")
evaluate(best_model)
print("First model...")
evaluate(first_model)

Best model...
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
10000/10000 [==============================] - 3s 317us/step
Model evaluation results: {'acc': 0.8474, 'loss': 0.5312415670394898}
First model...
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
10000/10000 [==============================] - 3s 296us/step
Model evaluation results: {'acc': 0.2053, 'loss': 2.0872473678588865}


## Try out your model on some manual inputs!

**Exercise**: write a digit into the canvas below. This will automatically load your input into the variable `final_data`. (Due to randomness, your mileage may vary. If your model is bad, try increasing the number of samples with `load_data(num_samples=X)` in the above defined trainables, or try the Optional Search Algorithm section below).

Typically, the model doesn't work quite well on 1 and 8s.

In [ ]:
HTML(open("input_final.html").read())

In [ ]:
manual_input = prepare_data(final_data)
best = best_model.predict(manual_input).argmax()
first = first_model.predict(manual_input).argmax()

print("Best model got {}, first model got {}".format(best, first))
plt.imshow(manual_input.reshape((28,28)))

# Congratulations, you're now a Tune expert!

# Please: fill out this form to provide feedback on this tutorial!

https://goo.gl/forms/NVTFjUKFz4TH8kgK2

# (Optional) Try using a search algorithm

Tune is an execution layer, so we can combine powerful optimizers such as HyperOpt (https://github.com/hyperopt/hyperopt) with state-of-the-art algorithms such as HyperBand without modifying any model training code.

The documentation to doing this is here: https://ray.readthedocs.io/en/latest/tune-searchalg.html#hyperopt-search-tree-structured-parzen-estimators

In [23]:
from hyperopt import hp
from ray.tune.suggest import HyperOptSearch

space = {
    "lr": hp.uniform("lr", 0.001, 0.1),
    "momentum": hp.uniform("momentum", 0.1, 0.9),
    "hidden": hp.choice("hidden", np.arange(16, 256, dtype=int)),
}

## TODO: CREATE A HyperOptObject
hyperopt_search = HyperOptSearch(space, reward_attr="mean_accuracy")

## TODO: Pass in the object to Tune.
good_results = tune.run_experiments(
    configuration2, search_alg=hyperopt_search, scheduler=hyperband, verbose=False)

2019-04-04 15:30:18,133	INFO tune.py:60 -- Tip: to resume incomplete experiments, pass resume='prompt' or resume=True to run()
2019-04-04 15:30:18,134	INFO tune.py:211 -- Starting a new experiment.


AttributeError: 'float' object has no attribute 'items'

In [ ]:
# Feel free to compare your best model here
best_model2 = get_best_model(make_model, good_results, metric="mean_accuracy")

In [ ]:
# Using the canvas data from above 

manual_input = prepare_data(final_data)
best = best_model.predict(manual_input).argmax()
best2 = best_model2.predict(manual_input).argmax()
first = first_model.predict(manual_input).argmax()

print("Best model got {}, HyperOpt model got {}, first model got {}".format(best, best2, first))
